## Optimal Power Flow
**Power Systems Optimization**

by Michael R. Davidson and Jesse D. Jenkins (last updated: September 27, 2020)

This notebook consists an introductory glimpse of and s few hands-on activities and demostrations of the Optimal Power Flow (OPF) problem—which minimizes the short-run production costs of meeting electricity demand from a given set of generators subject to various technical and flow limit constraints.

We will talk about a single-time period, simple generator, and line flow limit constraints (while modeling the network flows as dictated by the laws of physics). This is adds a layer of complexity and sophistication on top of the Economic Dispatch (ED) problem.

Since we will only discuss single time-period version of the problem, we will not be considering inter-temporal constraints, like ramp-rate limits. However, this model can easily be extended to allow for such constraints.

We will start off with some simple systems, whose solutions can be worked out manually without resorting to any mathematical optimization model and software. But, eventually we will be solving larger system, thereby emphasizing the importance of such software and mathematical models.

## Introduction to OPF
Every day, system operators need to decide how to meet demand from a large variety of generators with different costs and engineering requirements. Prior to large-scale computational capabilities, this was accomplished by "priority lists" or similar heuristics to determine the ordering of plants. However, with modern optimization algorithms, operators can find the global minimum of production costs—and even incorporate coupling of engineering processes across time periods.

Economic dispatch (ED) is the problem of minimizing short-run costs of production in order to meet a given demand, considering relevant engineering constraints of the generators. It does not fully capture the physics of electricity flows, which will further constrain the feasible space and which we will come back to in later notebooks on optimal power flow, nor does it account for decisions and constraints related to turning on or "committing" large thermal generators, which we'll discuss in our next notebook on unit commitment. Typically, it does incorporate some type of network representation, which we have ignored here by considering only a single bus. More on the networks in the notebook on optimal power flow.

## Single-time period, simple generator constraints
We will first examine the case where we are optimizing dispatch for a single snapshot in time, with only very simple constraints on the generators. $x^2$


$$
\begin{align}
		\mathbf{Objective\;Function:}\min_{P_g}\sum_{g\in{G}}C_{g}(P_{g})\longleftarrow\mathbf{power generation cost}\\
		\mathbf{Subject\;to:\:}{\underline{P}_{g}}\leqslant{P_{g}}\leqslant{{\overline{P}_{g}}},\;\forall{g\in{G}}\longleftarrow\mathbf{MW generation limits}\\
		P_{g(i)}-P_{d(i)}\longleftarrow\mathbf{real power injection}\notag\\=\sum_{j\in J(i)}B_{ij}(\theta_j-\theta_i),\;\forall{{i}\in\mathcal{N}}\\
		|P_{ij}|\leqslant{\overline{P}_{ij}},\;\forall{ij}\in{T}\longleftarrow\mathbf{MW line limit}\\
\end{align}
$$
 
The decision variable in the above problem is:

𝐺𝐸𝑁𝑔 , the generation (in MW) produced by each generator,  𝑔 
The parameters are:

𝑃𝑚𝑖𝑛𝑔 , the minimum operating bounds for the generator (based on engineering or natural resource constraints)
𝑃𝑚𝑎𝑥𝑔 , the maximum operating bounds for the generator (based on engineering or natural resource constraints)
𝐷𝑒𝑚𝑎𝑛𝑑 , the demand (in MW)
(For this simple problem, we will let  𝑃𝑚𝑖𝑛𝑔=0  and revisit this later in the unit commitment notebook.)

In addition, we have:

𝑉𝑎𝑟𝐶𝑜𝑠𝑡𝑔=𝑉𝑎𝑟𝑂𝑀𝑔+𝐻𝑒𝑎𝑡𝑅𝑎𝑡𝑒𝑔×𝐹𝑢𝑒𝑙𝐶𝑜𝑠𝑡𝑔
 
Note that, in contrast with the basic capacity expansion problem, we are not concerned with fixed costs. Why not?

The simplest response is that these costs have already been incurred and regardless of how much a generator produces, its fixed costs will not change. These costs are thus "sunk costs" and are constant in the objective function. Our optimal decision variables would not change by adjusting this constant up or down.

Therefore, we can safely ignore fixed costs for the purposes of finding optimal dispatch. (We will still have to consider them to calculate producer profits, however.)

A famous electricity regulator, noting that it is frequently misunderstood and ignored even by experienced market participants, has called the application of the "sunk cost fallacy" to electricity systems "Grandma's Inheritance Theorem." In this hypothetical scenario, if you were to inherit a diamond ring or perhaps a financial contract-for-differences that pays out relative to cleared market prices (you have a very sophisticated grandma in this scenario!), you should not be tempted to change your production strategy.

Now, let's implement ED.

In [12]:
1. Load packages¶

LoadError: syntax: extra token "Load" after end of expression

In [7]:
# New packages introduced in this tutorial (uncomment to download the first time)
# import Pkg; Pkg.add("PrettyTables"); Pkg.add("VegaLite")
using JuMP, GLPK
using Plots; plotly();
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1273
┌ Info: Precompiling GLPK [60bf3e95-4087-53dc-ae20-288a0d20c6a6]
└ @ Base loading.jl:1273
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1273
┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/samie/.julia/packages/Plots/jZRxE/src/backends.jl:373
┌ Info: Precompiling VegaLite [112f6efa-9a02-5b7d-90c0-432ed331239a]
└ @ Base loading.jl:1273
┌ Info: Precompiling DataFrames [a93c6f00-e57d-5684-b7b6-d8193f3e46c0]
└ @ Base loading.jl:1273
┌ Info: Precompiling CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1273
┌ Info: Precompiling PrettyTables [08abe8d2-0d0c-5749-adfa-8a2ac140af0d]
└ @ Base loading.jl:1273


In [8]:
2. Load and format data
We will use data based on San Diego Gas and Electric (SDG&E, via the PowerGenome data platform) plus a few neighboring generators, consisting of:

25 generators (including some clustering of smaller generators, and excluding behind-the-meter solar)
estimated hourly demand for 2020 (net load at the transmission substation level after subtracting 600MW of behind-the-meter solar from original demand)
variable generation capacity factors
estimated natural gas fuel costs

LoadError: syntax: extra token "Load" after end of expression

In [10]:
datadir = joinpath("OPF_data") 
# Note: joinpath is a good way to create path reference that is agnostic
# to what file system you are using (e.g. whether directories are denoted 
# with a forward or backwards slash).
gen_info = CSV.read(joinpath(datadir,"Gen118.csv"), DataFrame);
line_info = CSV.read(joinpath(datadir,"Tran118.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"Load118.csv"), DataFrame);

# Rename all columns to lowercase (by convention)
for f in [gen_info, line_info, loads]
    rename!(f,lowercase.(names(f)))
end

MethodError: MethodError: no method matching read(::String, ::Type{DataFrame})
You may have intended to import Base.read
Closest candidates are:
  read(::Any; copycols, kwargs...) at /home/samie/.julia/packages/CSV/GCUID/src/CSV.jl:1071

In [11]:
#=
Function to solve Optimal Power Flow (OPF) problem (single-time period)
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
Note: it is always a good idea to include a comment blog describing your
function's inputs clearly!
=#
function OPF_single(gen_df, line_info, loads)
    OPF = Model(GLPK.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # A set of all variable generators
    G_var = gen_df[gen_df[!,:is_variable] .== 1,:r_id] 
      # A set of all non-variable generators
    G_nonvar = gen_df[gen_df[!,:is_variable] .== 0,:r_id]
      # Set of all generators
    G = gen_df.r_id
    # Extract some parameters given the input data
      # Generator capacity factor time series for variable generators
    gen_var_cf = innerjoin(gen_variable, 
                    gen_df[gen_df.is_variable .== 1 , 
                        [:r_id, :gen_full, :existing_cap_mw]], 
                    on = :gen_full)
        
    # Decision variables   
    @variables(ED, begin
        GEN[G]  >= 0     # generation
        # Note: we assume Pmin = 0 for all resources for simplicty here
    end)
                
    # Objective function
    @objective(ED, Min, 
        sum( (gen_df[i,:heat_rate_mmbtu_per_mwh] * gen_df[i,:fuel_cost] +
            gen_df[i,:var_om_cost_per_mwh]) * GEN[i] 
                        for i in G_nonvar) + 
        sum(gen_df[i,:var_om_cost_per_mwh] * GEN[i] 
                        for i in G_var)
    )

    # Demand constraint
    @constraint(ED, cDemand, 
        sum(GEN[i] for i in G) == loads[1,:demand])

    # Capacity constraint (non-variable generation)
    for i in G_nonvar
        @constraint(ED, GEN[i] <= gen_df[i,:existing_cap_mw])
    end

    # Variable generation capacity constraint
    for i in 1:nrow(gen_var_cf)
        @constraint(ED, GEN[gen_var_cf[i,:r_id] ] <= 
                        gen_var_cf[i,:cf] *
                        gen_var_cf[i,:existing_cap_mw])
    end

    # Solve statement (! indicates runs in place)
    optimize!(ED)

    # Dataframe of optimal decision variables
    solution = DataFrame(
        r_id = gen_df.r_id,
        resource = gen_df.resource,
        gen = value.(GEN).data
        )

    # Return the solution and objective as named tuple
    return (
        solution = solution, 
        cost = objective_value(ED),
    )
end

OPF_single (generic function with 1 method)